#Otros datos:
https://www.epdata.es/datos/accidente-avion-datos-estadisticas/205?accion=2
muertos_en_accidente_de_avion_desde_1942.csv
numero_de_accidentes_aereos_desde_1942.csv


https://aviation-safety.net/database/


#Paises y ciudades
https://www.forosdelweb.com/f86/base-datos-paises-estados-ciudades-del-mundo-1157860/
https://docs.google.com/spreadsheets/d/1x-t5GdfxbeuO5GtfnX4BYbTC6XBRI8HgxUfHIclR8bw/edit#gid=565147435

#info geopy
#https://www.europeanvalley.es/noticias/geolocalizacion-y-mapas-con-python/

In [ ]:
#https://www.europeanvalley.es/noticias/geolocalizacion-y-mapas-con-python/

In [1]:
import pandas as pd 
from pandasgui import show
import re
from sklearn import preprocessing
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv('AccidentesAviones.csv')

In [3]:
#df.info()
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [3]:
# 1.Se verifica 'Unnamed: 0', se borra ya que podemos usar el mismo indice del df
if 'Unnamed: 0' in df.columns:
    df.drop(columns=['Unnamed: 0'],inplace=True)
    
#df['Unnamed: 0'] = df['Unnamed: 0'].astype('Int32')
#len(df['Unnamed: 0'].unique())

In [4]:
# 2. Se tranforma la columna fechas y HORA declarada
# Se corrigen las horas
#from time import  strftime

def convertir_dig_en_hora(elem):
    # Se verifica que los numeros coincidan con formato hora 
    if int(elem[:2]) < 24:
        h = elem[:2]
    else:
        h = "00"
    # Se verifica que los numeros coincidan con formato minutos 
    if int(elem[-2:]) < 60:
        min = elem[-2:]
    else:
        min = "00"
    return h+":"+min
    
def conversion_dataTime(pdSerie):
    hora = []
    for elem in pdSerie:
        if elem == "?":
        # Se asigna las 0000 a cualquier hora desconocida "?"
            hora.append("00:00")
        elif elem.isdigit(): 
        # Se verifica si son digitos que se puedan transformar a hora
            hora.append(convertir_dig_en_hora(elem))      
        else:
        # Si no son puros digitos se verifica si se pueden sacar al menos 4 digitos
            digitos = []
            for digito in list(elem):
                if digito.isdigit():
                    digitos.append(digito)
            if len(digitos) == 4:
                hora.append(convertir_dig_en_hora("".join(digitos)))
            else:
                hora.append("00:00")
    return hora
 
df["hora"] = conversion_dataTime(df['HORA declarada'])
df.insert(0, 'date', df['fecha']+" "+df["hora"])

#LA fecha no tienen problema porque pd.to_datetime las reconce automaticamente
df["date"] = pd.to_datetime(df["date"])

df.drop(columns=['HORA declarada',"fecha","hora"],inplace=True)


In [5]:
#Codificamos el Operador
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

df['OperadOR'].replace('?',"Desconocido",inplace=True)

df.insert(1, 'IdOperador', label_encoder.fit_transform(df['OperadOR']))

#df['IdOperador'] = label_encoder.fit_transform(df['OperadOR'])
df.rename(columns = {'OperadOR':'Operador'},inplace=True)


#Se crea la tabla Operadores donde se almacenara informacion relativa a los operadores:
Operadores =df.loc[:, ('IdOperador', 'Operador')]
Operadores.drop_duplicates(inplace=True)
Operadores.sort_values('IdOperador', ascending =True,inplace=True)
Operadores.reset_index(inplace=True)
Operadores.drop(columns=['index'],inplace=True)

#Agregamos una coluna para indicar si el operador es militar 
val_buscar = ['Air Force','Military', 'Fuerza Aerea', 'Fuerza Area', 'Fuerza Aérea']
Operadores["Militar"] = Operadores["Operador"].str.contains('|'.join(val_buscar) ,case = False,regex=True).astype('Int32') 
#Operadores.apply(lambda x: x["Militar"]==1 ,axis=1).sum()
#Operadores["Militar"].value_counts()

#Se elimina la columna del df
df.drop(columns=['Operador'],inplace=True)

In [63]:
#df2 = df[df['date'].dt.year > 2000]
#df2.shape

(697, 17)

In [8]:
#Codificamos la columna 'route' que es la ruta del vuelo 

#Se crearan las columnas Ruta_Pais_Origen, Ruta_ciudad_Origen,Ruta_Pais_Destino, Ruta_ciudad_Destino 
df['route'].replace('?',"No_identificado",inplace=True)

#Para hacer la prueba se toma una muestra del dataset despues se cambia df2 por df
df2 = df.sample(50)
df2 = df2.reset_index()
df2.drop(columns=['index'],inplace=True)

#Se separan los valores de la columna 'route' que debe contener las ciudades ruta del vuelo
separadores =['/','-',',']
s2 = df2['route'].str.split(pat='|'.join(separadores),expand=True)

#Opcion #1 usando geopy
#El objetivo es, dado una lista de "ciudades" verificar si son todas del mismo pais, de ser asi
# devolver un string con la lista de ciudades ordenadas de forma alfabetica indicando al final el pais con el codigo del mismo
# de no pertenecer al mismo pais devolver una lista de los paises a los que pertenecen esas ciudades
#  nota de tener un valor ["Training", "Demonstration","Air show","Test flight","No_identificado"] devuelve "No_identificado"

from IPython.display import clear_output
from geopy.geocoders import Nominatim
#import time
#time.sleep(.5)     #Hay que darle tiempo a que responda sino da error siempre

geolocator = Nominatim(user_agent = "geoapiExercises")

def buscar_paises(Pdserie):
    Pdserie = Pdserie.unique()
    descarte = ["Training", "Demonstration","Air show","Test flight","No_identificado"] 
    ciudades = []
    paises = []
    for ciudad in Pdserie:
        if ciudad  and ciudad not in descarte:
            ciudades.append(ciudad.strip())
            try:
                location = geolocator.geocode(ciudad)
                location = geolocator.reverse(location.raw['lat']+","+location.raw['lon'])
                paises.append(location.raw['address']['country_code'])  #['country_code']
            except:
                pass
    paises = list(set(paises))
    if len(paises) > 1:
        return ",".join(paises), 1
    elif len(paises) == 0:
        return "No_identificado",2
    else: 
        ciudades.sort()
        return ",".join(ciudades)+"-"+paises[0],0
    

resultado =[]
JHO = 0
JHO_max = s2.shape[0]
for i in range(s2.shape[0]):
    JHO = JHO + 1
    clear_output(wait=True)
    print('Completado: ' + str(round(JHO / JHO_max * 100, 2)) + '%')
    resultado.append(buscar_paises(s2.iloc[i,:]))


Vuelo_internacional = []
Ruta_vuelo = []
for i in range(len(resultado)):
    Vuelo_internacional.append(resultado[i][1])
    Ruta_vuelo.append(resultado[i][0])
    
df2.insert(2, 'Ruta_vuelo',Ruta_vuelo )
df2.insert(3, 'Tipo_vuelo_Nac_Int',Vuelo_internacional )

show(df2)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Completado: 100.0%


In [15]:
#Codificamos el lugar del accidente columna 'Ruta'

df['Ruta'].replace('?',"No_identificado",inplace=True)

r = df['Ruta'].str.split(pat=',',expand=True)

#r.shape[1]
r[r[r.shape[1]-1].notnull()]


,0,1,2,3
149,Florianopolis,Brazil\rFlorianopolis,Brazil\rFlorianopolis,Brazil
665,Near Cabo Ruivo,Marine Base,Lisbon,Portugal
1729,NAS Argentia,Placentia Bay,Newfoundland,Canada
1774,Near Varese,Lombardia,20 miles NW of Milan,Italy
2385,Gatwick Airport,Horley,Surrey,England
2617,London Heathrow,Staines,Surrey,England
2865,Moron AFB,Spain\r\t\rMoron AFB,Spain\r\t\rMoron AFB,Spain
3155,Mt. San Pietro,near Ajaccio,Corsica,France
3724,Djibouti City,Djibouti\r\tDjibouti City,Djibouti\rDjibouti City,Djibouti
3734,Near Point Alert,Ellesmere Island,NWT,Canada


In [16]:
r

,0,1,2,3
0,Fort Myer,Virginia,None,None
1,Juvisy-sur-Orge,France,None,None
2,Atlantic City,New Jersey,None,None
3,Victoria,British Columbia,Canada,None
4,Over the North Sea,None,None,None
...,...,...,...,...
5003,Near Butte,Alaska,None,None
5004,Near Kaduna,Nigeria,None,None
5005,Near Pyin Oo Lwin,Myanmar,None,None
5006,Patikul,Sulu,Philippines,None


In [29]:
i=5006
r.iloc[i,0]

'Patikul'

In [31]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent = "geoapiExercises")
location = geolocator.geocode(r.iloc[i,0])
location = geolocator.reverse(location.raw['lat']+","+location.raw['lon'])
location.raw['address']['country']  #['country_code']

'Philippines'

In [28]:

s1 = List_ciudades["CIUDAD"]
List_ciudades[s1.str.contains(r.iloc[i,0], flags=re.IGNORECASE, regex=True,na=False)]


,Unnamed: 0,ID,CODIGO PAIS,CIUDAD


In [ ]:

import re
val_buscar = ['PARROT','HOUSE']
s1 = pd.Series(['Mouse', 'dog', 'house and parrot', '23','parrot','52F47', 0])
s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False)

In [13]:
#Opcion 2, usando un listado de paises y ciudades
# Obtenido de:
# https://www.forosdelweb.com/f86/base-datos-paises-estados-ciudades-del-mundo-1157860/
# https://docs.google.com/spreadsheets/d/1x-t5GdfxbeuO5GtfnX4BYbTC6XBRI8HgxUfHIclR8bw/edit#gid=565147435


List_Paises = pd.read_csv('datos/PAISES.csv')
List_ciudades = pd.read_csv('datos/CIUDADES DE PAISES.csv')


In [43]:
List_ciudades.sample(5)

,Unnamed: 0,ID,CODIGO PAIS,CIUDAD
132918,NaN,132919,VN,Lang Ha
86446,NaN,86447,US,East Orland
61787,NaN,61788,BE,Baasrode
2844,NaN,2845,US,Kiowa
659,NaN,660,US,Highland


In [44]:
List_ciudades[List_ciudades["CIUDAD"] == 'Corsica']


,Unnamed: 0,ID,CODIGO PAIS,CIUDAD
80098,NaN,80099,US,Corsica


In [70]:
s3 = s2[0].str.split(pat="-",expand=True)

In [64]:
#s2[s2[2].isna()]
s2[s2[2].notnull()]

,0,1,2
4397,Moscow-Munich-Barcelona,Bahrain,Brussels


In [18]:
df2.apply(lambda x: x["OperadOR"] == '?', axis=1).sum()

0

In [20]:
df.apply(lambda x: x['Ruta'] == '?', axis=1).sum()

5

cn_ln
Construction or serial number / Line or fuselage number

In [15]:
df.apply(lambda x: x["OperadOR"] == '?', axis=1).sum()

10

In [ ]:
#Para obtener info de la ultima columna

import re
val_buscar = ['PARROT','HOUSE']
s1 = pd.Series(['Mouse', 'dog', 'house and parrot', '23','parrot','52F47', 0])
s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False)


a_string = "A string is more than its parts!"
matches = ["more", "string"]
if all(x in a_string for x in matches):
    print("Si estan todos")

a_string = "A string is more than its parts!"
matches = ["more", "wholesome", "milk"]
if any(x in a_string for x in matches):
    print("Si esta alguno")

In [11]:
import re
val_buscar = ['PARROT','HOUSE']
s1 = pd.Series(['Mouse', 'dog', 'house and parrot', '23','parrot','52F47', 0])
s1.str.contains('|'.join(val_buscar), flags=re.IGNORECASE, regex=True,na=False)

0    False
1    False
2     True
3    False
4     True
5    False
6    False
dtype: bool

#Obtencion de mas informacion

#Procedimiento de : https://www.youtube.com/watch?v=fOR4bLgR00I
Web Scraping - Extracción de datos web con Python 1


In [9]:
import requests
from bs4 import BeautifulSoup as b 
url = "https://aviation-safety.net/statistics/period/stats.php"
#url = "https://www.mercadolibre.com.mx/"
#url = "https://articulo.mercadolibre.com.mx/MLM-1386825775-inversor-de-corriente-coche-12v-a-110v-150w-1-ac-y-2-_JM#reco_item_pos=3&reco_backend=machinalis-homes-pdp-boos&reco_backend_type=function&reco_client=home_second-best-navigation-trend-recommendations&reco_id=fbbd5122-74d9-490e-b15a-d18b3637d43e&c_id=/home/second-best-navigation-trends-recommendations/element&c_element_order=4&c_uid=4d8503ef-e075-49ab-b1e6-f3052d2e6cf4"
#url = "https://articulo.mercadolibre.com.mx/MLM-929342413-maquina-de-pasta-para-batidora-de-soporte-kitchenaid-_JM?variation=87326131747#reco_item_pos=1&reco_backend=machinalis-homes-pdp-boos&reco_backend_type=function&reco_client=home_navigation-trend-recommendations&reco_id=6d8fea94-ae8d-4821-8094-2ebb00ee08d7&c_id=/home/navigation-trends-recommendations/element&c_element_order=2&c_uid=e1a5d46a-ae89-443a-957d-45a3e8e5a156"

html =requests.get(url)
content = html.content 
soup = b(content,"lxml")
print(soup)



ModuleNotFoundError: No module named 'requests'

In [ ]:
titulo1 = soup.find("h1",{"class":"ui-pdp-title"})
print(titulo1.text)
moneda1 = soup.find("span",{"class":"andes-money-amount__currency-symbol"})
precio1 = soup.find("span",{"class":"andes-money-amount__fraction"})
print(moneda1.text,precio1.text)
descuento = soup.find("span",{"class":"ui-pdp-price__second-line__label ui-pdp-color--GREEN ui-pdp-size--MEDIUM"})
print("Descuento %",descuento.text)